# Design Shorten URL service

### APIs
- How the user interacts with the service

```
    createShort(longURL) -> shortURL
    getLong(shortURL) -> longURL
```

### Application Layer
- **Client**
    + Connect to service by REST / HTTP
    + do API requests createShort() / getLong()

- Load balancer
    + Distribute client requests -> Worker hosts
- Worker threads/hosts
    + When receive a request, check if the <shortURL/longURL> in the distributed cache 1st. 
    + If not access DB:
        + createShort(longURL): Take longURL -> Generate tinyURL -> Store and cache both longURL and shortURL in database
        + getLong(shortURL): Take shortURL -> get the longURL from database -> cache and return to the client

- Distributed cache
    + memcache/redis
    + Store recent create/get URLs (maybe 2 hrs)
        + People tends to use recent creating URL than the one created long time ago
        
<img src="./img/1.jpg" alt="drawing" width="850"/>
     
### Design TinyURL

#### Shorten characters

```
    a-z: 26 chars
    A-Z: 26 chars
    0-9: 10 chars
        -> Total 62 chars
```

- If we choose 7 char length. Eg: bit.ly/8E4Acad
    + Number of available URL = $62^7=3.5E12$
    + If the service provide 1000 URL/second -> Take 110 years to exhaust
    + If the service provide 1000000 URL/second -> Just take 40 days to exhaust
    
- 128bit MD5 hash
    + $log_2(3.5E12)$ ~ 42: We can repesent the 7-char length URL by 42 bits

#### Database table schema
- <key, value> = <shortURL, longURL>

#### Techniques to generate tinyURL
1. Generate a random shortURL and check db

<img src="./img/2.jpg" alt="drawing" width="850"/>

2. Hash the long URL
    - md5hash(longURL) -> 128bit
    - If we choose 7 char length shortURL, pick the 1st 42bit
    - map 42bit -> 7 char length string
    - check DB and put <shortURL, longURL> like 1.
    
```
    Advantages:
        + reuse of shortURL resource if >2 clients want to shorten the same longURL 
```
    
```python
def createShortURL(longURL):
    bit_string_42 = md5Hash(longURL)[0:42]
    nums_based_62 = convert_to_base_62(bit_string_42)
    url = 'myShortenURL.com/'
    
    for each num in nums_based_62:
        if 0 <= num and num <= 26:
            c = 'a' - 'z'
        elif 27 <= num and num <= 53:
            c = 'A' - 'Z'
        else:
            c = '0' - '9'
            
        url += c
```

3. Counter base
    - provide shortURL increasingly based on ranges

<img src="./img/3.jpg" alt="drawing" width="850"/>

### Advanced
- getLong(shortURL) faster than createShort(longURL)
- Consider CDN for createShort(longURL) service